In [192]:
print("""
@File         : ch02_finding_and_wrangling_time_series_data.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-03-08 23:25:05
@Email        : cuixuanstephen@gmail.com
@Description  : 时间序列的发现与整理
""")


@File         : ch02_finding_and_wrangling_time_series_data.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-03-08 23:25:05
@Email        : cuixuanstephen@gmail.com
@Description  : 时间序列的发现与整理



In [193]:
import pandas as pd

In [194]:
year_joined = pd.read_csv('../data/year_joined.csv')

In [195]:
# 可以看出来每个用户只有一个账户
year_joined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [196]:
year_joined.groupby('user').count()

,userStats,yearJoined
user,,
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
995,1,1
996,1,1
997,1,1


In [197]:
emails = pd.read_csv('../data/emails.csv', parse_dates=['week'])

In [198]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [199]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08
25469,2.0,998.0,2018-01-15
25470,3.0,998.0,2018-01-22
25471,2.0,998.0,2018-01-29
25472,3.0,998.0,2018-02-05
25473,3.0,998.0,2018-02-12


可以看出来缺少了几周数据，从 2017 年 12 月 18 日之后，就没有该月份的任何邮件事件了。

In [200]:
(
    max(emails[emails.user == 998]["week"]) - min(emails[emails.user == 998]["week"])
).days / 7

25.0

In [201]:
emails[emails.user == 998].shape

(24, 3)

可以看出数据只有 24 条，但是应该有 26 条。

你可能会感到惊讶，考虑到我们刚刚执行的减法，我们需要 26 而不是 25，但这是一个不完整的计算。当你处理时间序列数据时，在进行这种减法之后，你应该始终问自己的一件事是是否应该加 1 。换句话说，你是否减去了你想要统计的位置？

有些确实数据是在有记录之前或者之后，我们没有办法进行填补。但是第一条记录和最后一条记录之间的数据非空数据可以补全。

In [202]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [203]:
all_email = (
    emails.set_index(["week", "user"])
    .reindex(complete_idx, fill_value=0)
    .reset_index(names=["week", "user"])
)
# all_email.columns = ["week", "user", "emailsOpened"]

In [204]:
all_email[all_email.user == 998].sort_values("week")

,week,user,emailsOpened
54438,2015-02-09,998.0,0.0
33956,2015-02-16,998.0,0.0
35573,2015-02-23,998.0,0.0
63062,2015-03-02,998.0,0.0
57672,2015-03-09,998.0,0.0
...,...,...,...
52282,2018-04-30,998.0,3.0
41502,2018-05-07,998.0,3.0
87856,2018-05-14,998.0,3.0
80849,2018-05-21,998.0,3.0


在全时间里面，有很多没有数据的空值，这些是没有必要的，因此在这里进行去除

In [205]:
emails.shape

(25488, 3)

In [206]:
cutoff_dates = emails.groupby("user")["week"].agg(["min", "max"]).reset_index()
cutoff_dates

,user,min,max
0,1.0,2015-06-29,2018-05-28
1,3.0,2018-03-05,2018-04-23
2,5.0,2017-06-05,2018-05-28
3,6.0,2016-12-05,2018-05-28
4,9.0,2016-07-18,2018-05-28
...,...,...,...
534,991.0,2016-10-24,2016-10-24
535,992.0,2015-02-09,2015-07-06
536,993.0,2017-09-11,2018-05-28
537,995.0,2016-09-05,2018-05-28


In [207]:
for _, row in cutoff_dates.iterrows():
    user = row["user"]
    start_date = row["min"]
    end_date = row["max"]
    # 这没有问题，因为 bool 类型的过滤会按照 index 匹配，但是不推荐
    # all_email = all_email.drop(
    #     all_email[all_email["user"] == user][all_email["week"] < start_date].index
    # )
    # all_email = all_email.drop(
    #     all_email[all_email["user"] == user][all_email["week"] > end_date].index
    # )
    mask = ((all_email["user"] == user) & (all_email["week"] < start_date)) | (
        (all_email["user"] == user) & (all_email["week"] > end_date)
    )
    # mask = (all_email["user"] == user) & (
    #     (all_email["week"] < start_date) | (all_email["week"] > end_date)
    # )
    all_email = all_email.drop(all_email[mask].index)

重采样必须将时间日期作为索引，这是必须的

In [224]:
donations = pd.read_csv("../data/donations.csv")
donations["timestamp"] = pd.to_datetime(donations["timestamp"])
donations = donations.set_index("timestamp")
agg_don = donations.groupby("user").apply(
    lambda df: df["amount"].resample("W-MON").sum().dropna()
)
agg_don = agg_don.reset_index()

In [225]:
agg_don

,user,timestamp,amount
0,0.0,2015-03-30,25.0
1,0.0,2015-04-06,0.0
2,0.0,2015-04-13,0.0
3,0.0,2015-04-20,0.0
4,0.0,2015-04-27,0.0
...,...,...,...
32347,995.0,2017-09-11,0.0
32348,995.0,2017-09-18,0.0
32349,995.0,2017-09-25,0.0
32350,995.0,2017-10-02,1000.0


In [266]:
merged_df = []
for user, user_email in all_email.groupby("user"):
    user_donations = agg_don[agg_don["user"] == user]

    user_donations = user_donations.set_index("timestamp")
    user_email = user_email.sort_values("week").set_index("week")

    df = pd.merge(
        user_email, user_donations, how="left", left_index=True, right_index=True
    )
    df = df.fillna(0)
    df["user"] = df["user_x"]
    merged_df.append(df.reset_index()[["user", "week", "emailsOpened", "amount"]])
merged_df = pd.concat(merged_df)

In [5]:
425 + 5504 - 490

5439